In [21]:
import numpy as np
import pandas as pd

In [22]:
temp_df = pd.read_csv("../data/IMDB Dataset.csv")

In [23]:
temp_df.shape

(50000, 2)

In [24]:
df = temp_df[:10000]

In [25]:
df.shape

(10000, 2)

In [26]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [27]:
df.drop_duplicates(inplace=True)

/tmp/ipykernel_8887/3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [28]:
df.duplicated().sum()

0

In [29]:
import re
def remove_tags(text):
    cleaned_text = re.sub(re.compile('<.*?>'),"",text)
    return cleaned_text

In [30]:
df['review'] = df['review'].apply(remove_tags)

/tmp/ipykernel_8887/2336150696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [31]:
df['review']

0       One of the other reviewers has mentioned that ...
1       A wonderful little production. The filming tec...
2       I thought this was a wonderful way to spend ti...
3       Basically there's a family where a little boy ...
4       Petter Mattei's "Love in the Time of Money" is...
                              ...                        
9995    Fun, entertaining movie about WWII German spy ...
9996    Give me a break. How can anyone say that this ...
9997    This movie is a bad movie. But after watching ...
9998    This is a movie that was probably made to ente...
9999    Smashing film about film-making. Shows the int...
Name: review, Length: 9983, dtype: object

In [32]:
df['review'] = df['review'].apply(lambda x:x.lower())

/tmp/ipykernel_8887/740760900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


0       one of the other reviewers has mentioned that ...
1       a wonderful little production. the filming tec...
2       i thought this was a wonderful way to spend ti...
3       basically there's a family where a little boy ...
4       petter mattei's "love in the time of money" is...
                              ...                        
9995    fun, entertaining movie about wwii german spy ...
9996    give me a break. how can anyone say that this ...
9997    this movie is a bad movie. but after watching ...
9998    this is a movie that was probably made to ente...
9999    smashing film about film-making. shows the int...
Name: review, Length: 9983, dtype: object

In [34]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

st_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /home/admi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in st_words]).apply(lambda x: " ".join(x))

/tmp/ipykernel_8887/647783855.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in st_words]).apply(lambda x: " ".join(x))


In [36]:
df['review']

0       one reviewers mentioned watching 1 oz episode ...
1       wonderful little production. filming technique...
2       thought wonderful way spend time hot summer we...
3       basically there's family little boy (jake) thi...
4       petter mattei's "love time money" visually stu...
                              ...                        
9995    fun, entertaining movie wwii german spy (julie...
9996    give break. anyone say "good hockey movie"? kn...
9997    movie bad movie. watching endless series bad h...
9998    movie probably made entertain middle school, e...
9999    smashing film film-making. shows intense stran...
Name: review, Length: 9983, dtype: object

In [42]:
import gensim

In [43]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [44]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /home/admi/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [45]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [47]:
model = gensim.models.Word2Vec(
    window = 10,
    min_count = 2
)

In [48]:
model.build_vocab(story)

In [49]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5875417, 6212140)

In [50]:
len(model.wv.index_to_key)

31845

In [53]:
def document_vector(doc):
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc],axis=0)

In [54]:
document_vector(df['review'].values[0])

array([-0.14467447,  0.5124344 ,  0.21408626,  0.25646666, -0.07475526,
       -0.541881  ,  0.23065318,  0.9673415 , -0.27636033, -0.29487023,
       -0.34721863, -0.4852526 ,  0.13073565,  0.14527094,  0.15181822,
       -0.2132148 ,  0.06198363, -0.35978153, -0.05265329, -0.6797001 ,
       -0.00515788,  0.17289919,  0.0906226 , -0.32865018, -0.27432677,
       -0.07959173, -0.29753837,  0.0424534 , -0.24501699,  0.02495354,
        0.29304516,  0.00252931,  0.25614953, -0.2550014 , -0.13571642,
        0.37478554,  0.15937473, -0.4034282 , -0.22785075, -0.7717985 ,
        0.10188301, -0.2080306 ,  0.01986943, -0.05964186,  0.51889074,
       -0.12286984, -0.2737578 ,  0.01501831,  0.12180257,  0.35489196,
        0.05361523, -0.31836444, -0.4468971 , -0.1626338 , -0.09577225,
        0.22020468,  0.24559759,  0.04086548, -0.3014081 ,  0.03569202,
        0.04314498,  0.12700255,  0.02748267, -0.07880753, -0.43899718,
        0.29138753, -0.02390308,  0.14226893, -0.40485758,  0.37

In [55]:
from tqdm import tqdm

In [56]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

 15%|█▌        | 1509/9983 [02:24<21:58,  6.43it/s]

In [ ]:
X = np.array(X)
X[0]